In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras import Sequential
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.shape

(768, 9)

In [4]:
X = df.drop(columns="Outcome").values
y = df['Outcome'].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [6]:
# scaling the data
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X_train_trf = scale.fit_transform(X_train)
X_test_tf = scale.transform(X_test)

# 1. Building a NN with random hyperparams

In [25]:
model = Sequential()
model.add(Input(shape=(8,)))
model.add(Dense(32, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# comiple the model
model.compile(loss="binary_crossentropy", optimizer = "adam", metrics = ['accuracy'])
history = model.fit(X_train_trf, y_train, validation_data = (X_test_tf, y_test), epochs = 10)

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.6615 - loss: 0.6551 - val_accuracy: 0.6823 - val_loss: 0.6200
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7083 - loss: 0.6040 - val_accuracy: 0.7135 - val_loss: 0.5949
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7378 - loss: 0.5641 - val_accuracy: 0.7240 - val_loss: 0.5763
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7569 - loss: 0.5338 - val_accuracy: 0.7292 - val_loss: 0.5643
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7674 - loss: 0.5087 - val_accuracy: 0.7240 - val_loss: 0.5556
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7778 - loss: 0.4910 - val_accuracy: 0.7188 - val_loss: 0.5501
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7830 - loss: 0.4768 - val_accuracy: 0.7292 - val_loss: 0.5476
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7865 - loss: 0.4658 - val_accuracy: 0.7292 - val_l

So, we are getting accuracy around 72 on validatation set

# 2. lets find the best hyperparameter values using kerastuner library

In [7]:
# installing the library
# %pip install keras-tuner

In [8]:
import keras_tuner as kt

In [15]:
# defining the hypermodel
def build_model(hp):
    model = keras.Sequential()

    model.add(Input(shape=(8,))) #input layer
    
    # TUNE: Number of hidden layers
    for i in range(hp.Int('num_layers', min_value=1, max_value=4)):
        model.add(Dense(
            # TUNE: Number of units in each layer
            units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
            # TUNE: Activation function
            activation=hp.Choice(f'activation_{i}', values=['relu', 'tanh'])
        ))
        
        # TUNE: Dropout rate (optional, but good for regularization)
        # We use hp.Boolean to decide if we want dropout or not
        if hp.Boolean(f'dropout_{i}'):
            model.add(Dropout(rate=0.25))

    # Output layer 
    model.add(Dense(1, activation='sigmoid'))

    # TUNE: Learning rate
    # generally commonly used log sampling [1e-2, 1e-3, 1e-4]
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    
    # TUNE: Optimizer choice
    optimizer_name = hp.Choice('optimizer', values=['adam', 'rmsprop'])
    
    if optimizer_name == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [16]:
# Instantiate the Tuner
# 'Hyperband' is an optimized version of random search that is very efficient
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',       # Folder to save logs
    project_name='intro_to_kt'
)

Reloading Tuner from my_dir\intro_to_kt\tuner0.json


In [17]:
# A call to stop training early if validation loss stops improving
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# 4. Run the Hyperparameter Search
tuner.search(X_train_trf, y_train, 
             epochs=10, 
             validation_split=0.2, 
             callbacks=[stop_early],
             batch_size=64) # You can also tune batch_size, but fixed is usually fine for a start


Trial 26 Complete [00h 00m 06s]
val_accuracy: 0.6724137663841248

Best val_accuracy So Far: 0.7844827771186829
Total elapsed time: 00h 02m 47s


In [27]:
# 5. Get the Best Hyperparameters
best_hps = tuner.get_best_hyperparameters()[0].values
for keys in best_hps:
    print(keys,best_hps[keys])

num_layers 2
units_0 224
activation_0 tanh
dropout_0 True
learning_rate 0.00704424116535504
optimizer adam
units_1 448
activation_1 tanh
dropout_1 False
units_2 96
activation_2 tanh
dropout_2 False
units_3 384
activation_3 relu
dropout_3 False
tuner/epochs 10
tuner/initial_epoch 0
tuner/bracket 0
tuner/round 0


In [26]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 224)                 │           2,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 224)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 448)                 │         100,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             449 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 309,797 (1.18 MB)

 Trainable params: 103,265 (403.38 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 206,532 (806.77 KB)

In [19]:
# 6. Build the model with the best hyperparameters and train it on full data
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_trf, y_train, epochs=10, validation_data=(X_test_tf,y_test))

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.7031 - loss: 0.6964 - val_accuracy: 0.7396 - val_loss: 0.5102
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7639 - loss: 0.5003 - val_accuracy: 0.7500 - val_loss: 0.5018
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7795 - loss: 0.4853 - val_accuracy: 0.7500 - val_loss: 0.5515
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7726 - loss: 0.4957 - val_accuracy: 0.7552 - val_loss: 0.5282
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7830 - loss: 0.4774 - val_accuracy: 0.7344 - val_loss: 0.5461
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7830 - loss: 0.4657 - val_accuracy: 0.7396 - val_loss: 0.4955
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7708 - loss: 0.4710 - val_accuracy: 0.7500 - val_loss: 0.5011
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7899 - loss: 0.4591 - val_accuracy: 0.7604 - v

here we can clearly see we are getting the max val accuracy of 76% which is more than earlier